## Updates
- Pytesseract attempt (done 23/6/2023)
- [Perspective Transformation (perspective)](https://towardsdatascience.com/perspective-versus-affine-transformation-25033cef5766)
- contrast adjustment
- update codebase for ocrimg and dashboard

In [47]:
%pip install opencv-python
%pip install matplotlib
%pip install pytesseract
%pip install regex
%pip install scipy

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Lim Hui Xuan\Desktop\coding\work\ocr\tessenv\Scripts\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Lim Hui Xuan\Desktop\coding\work\ocr\tessenv\Scripts\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Lim Hui Xuan\Desktop\coding\work\ocr\tessenv\Scripts\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Lim Hui Xuan\Desktop\coding\work\ocr\tessenv\Scripts\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Lim Hui Xuan\Desktop\coding\work\ocr\tessenv\Scripts\python.exe -m pip install --upgrade pip' command.


In [1]:
import cv2
import pytesseract
from PIL import Image
from matplotlib import pyplot as plt
import numpy as np
import imutils

In [2]:
def biggest_contour(contours):
    biggest = np.array([])
    max_area = 0
    for i in contours:
        area = cv2.contourArea(i)
        if area > 1000:
            peri = cv2.arcLength(i, True)
            approx = cv2.approxPolyDP(i, 0.15 * peri, True)
            print(approx)
            if area > max_area and len(approx) >= 4:
                biggest = approx
                max_area = area
    return biggest

def correct_skew(image):
    img = image.copy()
    
    #greyscale and search for edges
    #grayscale = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    grayscale = cv2.bilateralFilter(img, 20, 30, 30)
    edged = cv2.Canny(grayscale, 230, 240)
    
    #grayscale = np.stack((grayscale,) * 3, axis = -1)
    #edged = np.stack((edged,) * 3, axis = -1)
    #img_hor = np.hstack((img, grayscale, edged))
    
    # contour detection
    contours, hierachy = cv2.findContours(grayscale.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key = cv2.contourArea, reverse = True)[:1] # largest contour area
    biggest = biggest_contour(contours)
    
    # testing
    cv2.drawContours(img, [biggest], -1, (0, 255, 0), 3)
    
    # pixel values of the contours in the image
    points = biggest.reshape(4, 2)
    input_points = np.zeros((4, 2), dtype = "float32")
    
    points_sum = points.sum(axis = 1)
    input_points[0] = points[np.argmin(points_sum)]
    input_points[3] = points[np.argmax(points_sum)]
    
    points_diff = np.diff(points, axis = 1)
    input_points[1] = points[np.argmin(points_diff)]
    input_points[2] = points[np.argmax(points_diff)]
    
    (top_left, top_right, bottom_right, bottom_left) = input_points
    bottom_width = np.sqrt(((bottom_right[0] - bottom_left[0]) ** 2) + (bottom_right[1] - bottom_left[1]) ** 2)
    top_width =  np.sqrt(((top_right[0] - top_left[0]) ** 2) + (top_right[1] - top_left[1]) ** 2)
    right_height =  np.sqrt(((top_right[0] - bottom_right[0]) ** 2) + (top_right[1] - bottom_right[1]) ** 2)
    left_height =  np.sqrt(((top_left[0] - bottom_left[0]) ** 2) + (top_left[1] - bottom_left[1]) ** 2)
    
    #output image size
    max_width = max(int(bottom_width), int(top_width))
    max_height = max(int(right_height), int(left_height))
    
    #desired points
    converted_points = np.float32([[0, 0], [max_width, 0], [0, max_height], [max_width, max_height]])
    
    # perform the perspective transformation
    matrix = cv2.getPerspectiveTransform(input_points, converted_points)
    img_output = cv2.warpPerspective(img, matrix, (max_width, max_height))
    
    return img_output
    

In [3]:
# attempt new code for correcting perspective distortion
def order_points(pts):
	# initialize a list of coordinates that will be ordered
	# such that the first entry in the list is the top-left,
	# the second entry is the top-right, the third is the
	# bottom-right, and the fourth is the bottom-left
	rect = np.zeros((4, 2), dtype = "float32")
 
	# the top-left point will have the smallest sum, whereas
	# the bottom-right point will have the largest sum
	s = pts.sum(axis = 1)
	rect[0] = pts[np.argmin(s)]
	rect[2] = pts[np.argmax(s)]
 
	# now, compute the difference between the points, the
	# top-right point will have the smallest difference,
	# whereas the bottom-left will have the largest difference
	diff = np.diff(pts, axis = 1)
	rect[1] = pts[np.argmin(diff)]
	rect[3] = pts[np.argmax(diff)]
 
	# return the ordered coordinates
	return rect

def four_point_transform(image, pts):
    # obtain a consistent order of the points and unpack them
	# individually
	rect = order_points(pts)
	(tl, tr, br, bl) = rect
	# compute the width of the new image, which will be the
	# maximum distance between bottom-right and bottom-left
	# x-coordiates or the top-right and top-left x-coordinates
	widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
	widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
	maxWidth = max(int(widthA), int(widthB))
	# compute the height of the new image, which will be the
	# maximum distance between the top-right and bottom-right
	# y-coordinates or the top-left and bottom-left y-coordinates
	heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
	heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
	maxHeight = max(int(heightA), int(heightB))
	# now that we have the dimensions of the new image, construct
	# the set of destination points to obtain a "birds eye view",
	# (i.e. top-down view) of the image, again specifying points
	# in the top-left, top-right, bottom-right, and bottom-left
	# order
	dst = np.array([
		[0, 0],
		[maxWidth - 1, 0],
		[maxWidth - 1, maxHeight - 1],
		[0, maxHeight - 1]], dtype = "float32")
	# compute the perspective transform matrix and then apply it
	M = cv2.getPerspectiveTransform(rect, dst)
	warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))
	# return the warped image
	return warped


In [24]:
def correct_skew1(img):
    # load the image and compute the ratio of the old height
    # to the new height, clone it, and resize it
    ratio = img.shape[0] / 500.0
    orig = img.copy()
    img = imutils.resize(img, height = 500)
    
    # convert the image to grayscale, blur it, and find edges
    # in the image
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5, 5), 0)
    
    # manually calculate the threshold for the edge using median
    
    v, sigma = np.median(gray), 0.33

    #---- apply automatic Canny edge detection using the computed median----
    lower = int(max(0, (1.0 - sigma) * v))
    upper = int(min(255, (1.0 + sigma) * v))
    edged = cv2.Canny(gray, lower, upper)
    
    # find the contours in the edged image, keeping only the
    # largest ones, and initialize the screen contour
    cnts = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    cnts = sorted(cnts, key = cv2.contourArea, reverse = True)[:4]
    # loop over the contours
    for c in cnts:
        # approximate the contour
        cv2.drawContours(img, [c], -1, (0, 255, 0), 2)
        cv2.imshow("Outline", img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.02 * peri, True)
        # if our approximated contour has four points, then we
        # can assume that we have found our screen
        if len(approx) == 4:
            screenCnt = approx
            #break
    # show the contour (outline) of the piece of paper
    #print("STEP 2: Find contours of paper")
    cv2.drawContours(img, [screenCnt], -1, (0, 255, 255), 2)
    
    return img


In [25]:
def preprocess_img(img):
    #color to greyscale
    grayscale = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    #normalize image
    norm_img = np.zeros((img.shape[0], img.shape[1]))
    norm = cv2.normalize(grayscale, norm_img, 0, 255, cv2.NORM_MINMAX)
    blur = cv2.medianBlur(norm, 5)
    canny = cv2.Canny(blur, 40, 50)
    
    # denoise the image
    #img = cv2.fastNlMeansDenoisingColored(norm, None, 10, 10, 7, 15)
    
    #thresh = cv2.threshold(grayscale, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
    #dist = cv2.distanceTransform(thresh, cv2.DIST_L2, 5)
    #dist = cv2.normalize(dist, dist, 0, 1.0, cv2.NORM_MINMAX)
    #dist = (dist * 255).astype("uint8")

    return canny

In [26]:
# to test the code
test_path = "./test/096.jpg_Canon.jpg"
proc_img = correct_skew1(cv2.imread(test_path))

#cv2.namedWindow('result', cv2.WINDOW_NORMAL)
#cv2.imshow("result", proc_img)
#cv2.waitKey(0)

print(pytesseract.image_to_string(proc_img))
#print(pytesseract.image_to_string(Image.open(test_path)))

APPOIN

Apr - Oct
pr- Oct

650.322.8882
LajolieNailSpa.com

IBRUNABLE TO KEEP THE APPOINTME



In [1]:
import regex as re

def extract_text(img_path, lang):
    return pytesseract.image_to_string(Image.open(img_path), lang = lang)

def extract_name(ocr_result):
    # Define pattern to extract company
    company_pattern = r'(?i)(?:^|\b)([A-Z][A-Za-z]+(?: [A-Z][A-Za-z]+)*)(?:\b|$)'

    # Extract company using regular expression pattern
    company_match = re.search(company_pattern, ocr_result)
    if company_match:
        return company_match.group(1)
    else:
        return None

def extract_email(ocr_result):
    # Extract email address using regular expression pattern
    email_pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
    email_match = re.search(email_pattern, ocr_result, re.I)
    if email_match:
        return email_match.group()
    else:
        return None

def extract_company(ocr_result):
    # TODO: Define pattern to extract company
    company_pattern = r'(?i)(?:^|\b)([A-Z][A-Za-z]+(?: [A-Z][A-Za-z]+)*)(?:\b|$)'

    # Extract company using regular expression pattern
    company_match = re.search(company_pattern, ocr_result)
    if company_match:
        return company_match.group(1)
    else:
        return None

def extract_contact(ocr_result):
    # Extract contact number using regular expression pattern
    contact_number_pattern = r'\b\d+\b'
    contact_number_matches = re.findall(contact_number_pattern, ocr_result)
    contact_number = ''.join(contact_number_matches)
    if contact_number:
        return contact_number
    else:
        return None

In [25]:
# extract functions
extract_company(extract_text(test_path, lang = "eng"))

'Ale iS Beckman'

In [17]:
extract_name(extract_text(test_path, lang = "eng"))

'Alexis Beckman'

In [18]:
extract_email(extract_text(test_path, lang = "eng"))

'info@culturefrozenyogurt.com'

In [19]:
extract_contact(extract_text(test_path, lang = "eng"))

'6503240440650324044834094306100'

In [20]:
import json

# function to produce the json
def extract_json(img_path, lang):
    res_dict = {}
    
    extracted_txt = extract_text(img_path, lang = lang)
    res_dict["name"], res_dict["email"], res_dict["contact"], res_dict["company"] = extract_name(extracted_txt), extract_email(extracted_txt), extract_contact(extracted_txt), extract_contact(extracted_txt)
    
    res_json = json.dumps(res_dict)
    return res_json
    

In [21]:
# testing the json function
extract_json(test_path, "eng")

'{"name": "Alexis Beckman", "email": "info@culturefrozenyogurt.com", "contact": "6503240440650324044834094306100", "company": "6503240440650324044834094306100"}'